<a href="https://colab.research.google.com/github/gabrielcerono/GlioblastomaMultiforme/blob/main/Shieh2020_binary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sklearn as sk
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning) 
from imblearn.ensemble import BalancedBaggingClassifier
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn import metrics
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import matthews_corrcoef
import RFE_Glioblastomav2 as rfev2

In [4]:
dataset = pd.read_excel('/content/shieh.xlsx')

In [5]:
dataset.head(5)

,gender,age,statusPFS,statusOS,surgery,dose,TMZ,Chemogroup,Chemo,diagnostic year,OS,PFS,volume,gender.1,age.1
0,1,63,1,1,1,6000,1.0,1,1,2,1.36,1.260274,731.0,1,63
1,1,64,1,1,0,6600,0.0,2,1,1,8.84,5.115068,NaN,1,64
2,1,62,1,1,1,6000,1.0,1,1,2,1.46,0.641096,215.0,1,62
3,1,71,1,1,1,6000,NaN,3,1,1,2.42,0.652055,255.0,1,71
4,1,21,1,1,0,5940,0.0,2,1,2,1.26,1.079452,154.0,1,21


There are missing values

In [6]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

In [7]:
imp_mean = IterativeImputer(random_state=0)
data = imp_mean.fit_transform(dataset)

In [8]:
data = pd.DataFrame(data=data, columns= dataset.columns,)

In [ ]:
data.head()

In [10]:
data.corr(method = 'pearson')

,gender,age,statusPFS,statusOS,surgery,dose,TMZ,Chemogroup,Chemo,diagnostic year,OS,PFS,volume,gender.1,age.1
gender,1.000000,-0.163487,-0.139923,-0.113228,0.169842,0.117790,0.159428,-0.153072,-0.030024,-0.037493,0.043419,0.080638,-0.073217,1.000000,-0.163487
age,-0.163487,1.000000,0.202956,0.075994,-0.005687,-0.199397,0.046168,0.037592,0.110030,0.089632,-0.188313,-0.254452,0.153293,-0.163487,1.000000
statusPFS,-0.139923,0.202956,1.000000,0.570352,-0.063372,0.014962,-0.091558,0.045091,-0.044811,-0.075545,-0.022415,-0.143078,0.011984,-0.139923,0.202956
statusOS,-0.113228,0.075994,0.570352,1.000000,-0.111111,0.026233,-0.160530,0.079058,-0.078567,-0.132453,-0.028040,-0.045293,-0.083594,-0.113228,0.075994
surgery,0.169842,-0.005687,-0.063372,-0.111111,1.000000,-0.033926,0.171413,0.020329,0.235702,0.132453,-0.118900,-0.001697,0.088748,0.169842,-0.005687
dose,0.117790,-0.199397,0.014962,0.026233,-0.033926,1.000000,-0.132206,0.315258,0.297829,-0.334150,0.468474,0.378734,-0.162929,0.117790,-0.199397
TMZ,0.159428,0.046168,-0.091558,-0.160530,0.171413,-0.132206,1.000000,-0.730626,0.144294,0.515709,0.092544,0.077335,-0.057429,0.159428,0.046168
Chemogroup,-0.153072,0.037592,0.045091,0.079058,0.020329,0.315258,-0.730626,1.000000,0.570208,-0.370692,0.002759,-0.051486,0.013387,-0.153072,0.037592
Chemo,-0.030024,0.110030,-0.044811,-0.078567,0.235702,0.297829,0.144294,0.570208,1.000000,0.083252,0.115345,0.018430,-0.049697,-0.030024,0.110030
diagnostic year,-0.037493,0.089632,-0.075545,-0.132453,0.132453,-0.334150,0.515709,-0.370692,0.083252,1.000000,-0.345445,-0.203701,-0.077207,-0.037493,0.089632


In [11]:
data.drop(columns = ['TMZ', 'gender.1', 'age.1',], axis = 1, inplace= True)

In [12]:
data.columns

Index(['gender', 'age', 'statusPFS', 'statusOS', 'surgery', 'dose',
       'Chemogroup', 'Chemo', 'diagnostic year', 'OS', 'PFS', 'volume'],
      dtype='object')

## Binary Analysis on multiple classificators

In [25]:
X = data.drop(columns=['statusPFS', 'statusOS', 'PFS'], axis = 1)
y = data['statusOS']

In [15]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier

### Random Forest

In [26]:
clf = RandomForestClassifier(random_state=0)


The splitting was giving me problems in the ROC curve, as it didn't get any survival in the splitting. I had to stratify

In [28]:
mttavg= []
favg= []
accuracyavg = []
tpavg = []
tnavg = []
rocavg = []
praucavg = []




for x in range(100): 
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify = y)
  clf.fit(X_train, y_train)
  #That's for the ranking, now we gotta do for the validation set and also for all the metrics
  mttaverage = matthews_corrcoef(y_test, clf.predict(X_test))
  mttavg.append(mttaverage)
   #f1 score
  faverage = f1_score(y_test, clf.predict(X_test))
  favg.append(faverage)
  #accuracy score
  accuracyaverage = accuracy_score(y_test, clf.predict(X_test))
  accuracyavg.append(accuracyaverage)
  #TP and TN rate 
  tn, fp, fn, tp = confusion_matrix(y_test, clf.predict(X_test), labels =[0,1]).ravel()
  tprate =  tp / (tp + fn)
  tnrate = tn / (tn + fp)
  tpavg.append(tprate)
  tnavg.append(tnrate)
  # ROC AUC 
  roc = roc_auc_score(y_test, clf.predict_proba(X_test)[:, 1])
  rocavg.append(roc)
  #Precision recall area under de curve
  precision, recall, _thresholds = metrics.precision_recall_curve(y_test, clf.predict_proba(X_test)[:, 1])
  prauc = metrics.auc(recall, precision)     
  praucavg.append(prauc)


mttavgmean = np.mean(np.array(mttavg))
favgmean = np.mean(np.array(favg))
accuracyavgmean = np.mean(np.array(accuracyavg))
tpavgmean = np.mean(np.array(tpavg))
tnavgmean = np.mean(np.array(tnavg))
rocavgmean = np.mean(np.array(rocavg))
praucavgmean = np.mean(np.array(praucavg))
metricss = np.array([mttavgmean, favgmean, accuracyavgmean, tpavgmean, tnavgmean, rocavgmean, praucavgmean])
metricss = metricss.reshape(1,7)
metric_final = pd.DataFrame(data = metricss, columns = ['matthewscore', 'f1-score', 'Accuracy', 'True Positives', 'True Negatives', 'ROC AUC', 'PR AUC'])

In [29]:
metric_final

,matthewscore,f1-score,Accuracy,True Positives,True Negatives,ROC AUC,PR AUC
0,-0.0004,0.980188,0.961154,0.9996,0.0,0.5292,0.974781
